In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.losses import *
from keras.preprocessing.image import ImageDataGenerator
import skimage.io as io
import skimage.transform as tr
import skimage.color
from glob import glob

## Own Scripts
import helper
import loss_metrics 
from unet import UNet


In [ ]:
# Initialise variables
train_size = 18300
val_size = 3922
batch_size = 32

In [ ]:
# create generators for training 

trainInputPath = '/home/dan/Desktop/Datenset_TowerBuilding/train/images'
trainOutputPath = '/home/dan/Desktop/Datenset_TowerBuilding/train/masks'

valInputPath = '/home/dan/Desktop/Datenset_TowerBuilding/validate/images'
valOutputPath = '/home/dan/Desktop/Datenset_TowerBuilding/validate/masks'

trainGen = loadData(trainInputPath, trainOutputPath, batch_size)
valGen   = loadData(valInputPath, valOutputPath, batch_size)


In [ ]:
# Neues Modell initialisieren
model = None
model = UNet((224,224,1), 1, 16, 4, 2.0)
model.summary()

In [ ]:
#Altes Modell laden 
from keras.models import load_model 
model = load_model('ds_step08.h5', custom_objects={'iou_loss': iou_loss})
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss=iou_loss, metrics=['accuracy'])

In [ ]:
# Train and validate the model 
result = model.fit_generator(trainGen, steps_per_epoch= train_size/batch_size, epochs=10, validation_data=valGen, validation_steps=val_size/batch_size, verbose=1, shuffle=True)
print (result.history)

In [ ]:
model.save('/home/dan/Desktop/Datenset_TowerBuilding/results/training01.h5')

In [ ]:
# list all data in history

print(result.history.keys())
# summarize history for accuracy
plt.plot(result.history['acc'])
plt.plot(result.history['val_acc'])
plt.title('model accuracy') #Benennung des Grafen
plt.ylabel('accuracy') #Benennung der y-Achse
plt.xlabel('epoch')    #Benennung der x-Achse
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('model loss') #Benennung des Grafen
plt.ylabel('loss')  #Benennung der y-Achse
plt.xlabel('epoch') #Benennung der x-Achse
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Evaluate on the test dataset
testInputPath = '/home/dan/Desktop/Datenset_Tower/test/images'
testOutputPath = '/home/dan/Desktop/Datenset_Tower/test/masks'

testGen = loadData(testInputPath, testOutputPath, batch_size)

In [ ]:
model.evaluate_generator(testGen, steps=3922 ,verbose=1 )

In [ ]:
predict = model.predict_generator(testGen, steps=, verbose=1)

In [ ]:
b = 1
i = 85
plt.imshow(predict[i, ..., 0], cmap = 'gray')
plt.show()
#plt.imshow(testGen[b][i, ..., 0], cmap='gray')
#plt.show()